# Generate Bigram sentences based on Jane Austen books

### Notes

No NLP Packages

###  Import Required Packages to support project

In [2]:
from re import sub
import io
import pandas as pd
from collections import defaultdict
import random

###  Import Data

In [3]:
## Establish a variable with the file path
Collection = "/Users/man/GitHub/symmetrical-barnacle/data/JaneAusten_Collection.txt"

## Data provided by Project Gutenburg 
## Utilize with open to open the variable titles in a read only mode in utf-8 encoding and establish  
## a new variable then read it in / replace periods with the word 'PERIOD', once read-in, close the file
with io.open(Collection,'r',encoding='windows-1252') as repo:
    data = repo.read().replace('.',' PERIOD').split()

### Create pairs of words

In [4]:
## Create variables that contain lists in different portions of iteration (word 2 by offsetting word 1)
## this creates word pairs
word1 = list(data)
word2 = list(data[1:])

## Create a variable that is a list of lists
word_pair = list(zip(word1,word2))

### Create and clean up dataframes

In [5]:
## Establsh a dataframe in order to add 
word_df = pd.DataFrame()

## adding blank to the word 2 list as to ensure they are equal length for Pandas transformation
word2.append('NaN')

## Create two columns within the new word_df dataframe
word_df.loc[:,'word1'] = word1
word_df.loc[:,'word2'] = word2

## Establish a data frame called word1_df
## Reset the dataframe index
word1_df = pd.DataFrame()
word1_df = word_df.groupby('word1').size().reset_index(name='counts')

## Establish a data frame called pair_df
## Reset the dataframe index
pair_df = pd.DataFrame()
pair_df = word_df.groupby(['word1','word2']).size().reset_index(name='counts')

### Normalize / Generate Probabilities

In [6]:
## Clean up the word1 dataframe by renaming counts to add more specificity
word1_df.rename(columns={'counts':'word1counts'},inplace=True)
word1total = word1_df['word1counts'].sum()

## Create a new column/calculated field that establishs probablility by dividing word1 counts / word1 totals
word1_df['w1Probability'] = word1_df['word1counts'] / word1total

## Utilize Pandas merge to combine the pair_df and word1_df with the key being word1
mergedcount_df = pd.merge(pair_df,word1_df,on='word1')

## Establish a new column/calculated field called 'setProbability' that is a result of counts / word1 counts
mergedcount_df['setProbability'] = mergedcount_df['counts'] / mergedcount_df['word1counts']

### Generate 50 sentences based on a random word start

In [7]:
## Establsh variables
## word utilizes the bigram confusion matrix to chose the most probable word to start with (starting off)
## sentence variable establishes a list
word = random.choices(list(word1_df['word1']),weights=word1_df['w1Probability'])
sentence = []

In [9]:
## TESTING
mergedcount_df.loc[(mergedcount_df['word1']== 'difficulty')].head()

##Current issue with inner loop transition / 


,word1,word2,counts,word1counts,w1Probability,setProbability
96162,difficulty,PERIOD,7,82,0.000101,0.085366
96163,difficulty,about,3,82,0.000101,0.036585
96164,difficulty,allowed,1,82,0.000101,0.012195
96165,difficulty,and,1,82,0.000101,0.012195
96166,difficulty,"arising,",1,82,0.000101,0.012195


In [ ]:
## Loop that is built of two components: an internal loop that creates a sentence group
## Ensures that there are 50 iterations of sentences
for outerloop in range(0,50):
    ## Add inner loop to perform sentence creation and allow outer loop to control the number of sentences
    for innerloop in range(0,20):
        ## Add the variable words contents to the sentence list
        sentence.append(word)
        ## Establish a variable that utilizes 
        matchlike = mergedcount_df.loc[(mergedcount_df['word1']== word),['word2','setProbability']]
        word = random.choices(list(matchlike['word2']),weights=matchlike['setProbability'])[0]
        ## An if loop that ends an iteration of the inner loop if a return function is the next logical end
        ## re-establish the word variable based on the word one weights 
        if word == '/n':
            break
            word = random.choices(list(word1_df['word1']),weights=word1_df['w1Probability'])

In [ ]:
## Establish variable that is the output of the sentence list but clean and proces the result
## before printing the result set
revised = str(sentence).replace('PERIOD','.').replace(',','').replace('\'','').replace('\"','').replace(']','').replace('[','')
print(revised,'.')